# Backtest

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta
import time

import warnings
warnings.simplefilter('ignore')

In [ ]:
from technicals import TechnicalAnalysis
import data.marketdata.alpaca as api

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tickers = ['SQ','SSTK','ATVI','GOOG','SYK','VOD','OLN','ARE', 'DIS', 'C']
api_key_path = '../resources/api_keys.env'

In [ ]:
ohlcv = api.ohlcv(tickers=tickers, api_key_path=api_key_path)

In [ ]:
technicals = TechnicalAnalysis(ohlcv)

In [ ]:
print(f"Open price data for {technicals.tickers()[0]}")
technicals._open(ticker=technicals.tickers()[0]).tail(14).plot(color='black')

### *Backtesting Williams %R*

#### Test class method

In [ ]:
#set backtest bounds logic
wr_rules = {'long' : -80,
             'short' : -70}

wr_range = 10

# set cash position and tracker
seed_cash = 1000000.0

In [ ]:
backtest_test = technicals.wr_backtest(ticker = 'SQ',
                                       start_date = '2018-01-01',
                                       end_date = '2021-05-01',
                                       wr_rules = wr_rules,
                                       seed_cash = seed_cash,
                                       wr_range=wr_range)

In [ ]:
backtest_test.head()

In [ ]:
backtest_test[['Hold','Trade']].plot()

### *Backtest study*

In [ ]:
start_date = '2018-01-01'
end_date = '2021-05-01'

In [ ]:
# long_params = list(range(-85,-25, 3))
long_params = [-81, -80, -79]

long_params[:10]

In [ ]:
# short_params = list(range(-45,-75, -3))
short_params = [-69, -70, -71]

short_params[:10]

In [ ]:
# wr_ranges = [5,10,15,20,30,60,90,180]
wr_ranges = [5, 10, 15]

wr_ranges

In [ ]:
# ticker_paths : dict() of str : Path
# exchange = 'nyse'
# tickers_df = pd.read_csv(os.path.join('data','tickers', str(exchange + '.csv')))

tickers = ['DIS', 'C']

# tickers_df = pd.read_csv('data/tickers/finviz-midcap-PE_growth.csv')
# tickers = tickers_df["Ticker"].to_list()


In [ ]:
def backtest_wr(path):

    # instantiate data dump dictionary
    data_dump = {}
    
    # csv header bool
    first_time = True

    for ticker in tickers:
        print(f"Backtesting : {ticker}")

        # get stock data and clean up df
        ohlcv = api.ohlcv(tickers=[ticker],
                          api_key_path=api_key_path,
                          start_date=start_date,
                          end_date=end_date)
        
        # instantiate a TechnicalAnalysis class
        stock = TechnicalAnalysis(ohlcv)
        
        for wr_range in wr_ranges:
            for long in long_params:
                for short in short_params:
                    
                    try:
                        result_df = pd.DataFrame(columns=['ticker',
                                          'wr_range',
                                          'long_trigger',
                                          'short_trigger',
                                          'hold_return_pct',
                                          'trade_return_pct', 
                                          'delta_return',
                                          'delta_return_pct_diff'])

                        # set backtest bounds logic
                        wr_rules = {'long' : long,
                                     'short' : short}

                        # set cash position and tracker
                        seed_cash = 1000000.0

                        # set backtest parameters
                        data = stock.wr_backtest(ticker=ticker,
                                                 wr_rules = wr_rules,
                                                 seed_cash = seed_cash,
                                                 wr_range = wr_range,
                                                 start_date = start_date,
                                                 end_date = end_date)

                        init_hold_shares = data.iloc[0]['cash']/data.iloc[0]['close']

                        # parse data
                        data = data.set_index('time')

                        # process results data
                        hold_return = (data['Hold'][-1]-data['Hold'][0])/data['Hold'][0]
                        trade_return = (data['Trade'][-1]-data['Trade'][0])/data['Trade'][0]
                        delta_return = trade_return - hold_return
                        pct_delta_return = (trade_return - hold_return)/hold_return


                        result = {'ticker' : ticker,
                                  'wr_range' : wr_range,
                                  'long_trigger' : long,
                                  'short_trigger' : short,
                                  'hold_return_pct' : hold_return,
                                  'trade_return_pct' : trade_return,
                                  'delta_return' : delta_return,
                                  'delta_return_pct_diff' : pct_delta_return}

                        result_df = result_df.append(result, ignore_index=True)

                        if first_time:
                            result_df.to_csv(path, mode='w')
                            first_time = False
                        else:
                            result_df.to_csv(path, mode='a', header=False)
                            
                        print(f"Could not process {ticker} -> W%R range = {wr_range}; long bound = {long}; short bound = {short}")
                        
                    except:
                        pass

        
#     return result_df
        

In [ ]:
_path = 'data/results/2021_05_27-test1.csv'

In [ ]:
backtest_wr(_path)

In [ ]:
df = pd.read_csv(_path)

In [ ]:
df.head()

In [ ]:
print(len(df.index))

In [ ]:
# df.to_csv('2021_05_25-backtest-midcap-PE_growth.csv')

In [ ]:
filtered_df = df[df['delta_return_pct_diff'] > 1]
filtered_df

In [ ]:
len(filtered_df.index)/len(df.index)*100

In [ ]:
filtered_df.describe()

In [ ]:
filtered_df.delta_return_pct_diff.sort_values().plot.hist()

In [ ]:
plt.scatter(filtered_df.delta_return_pct_diff, filtered_df.wr_range)

In [ ]:
plt.scatter(filtered_df.delta_return_pct_diff, filtered_df.long_trigger)

In [ ]:
plt.scatter(filtered_df.delta_return_pct_diff, filtered_df.short_trigger)